### 손실함수

In [4]:
import numpy as np

In [5]:
# 오차제곱합

def sum_of_squares(y, t):
    return 0.5 * np.sum((y-t)**2)

In [11]:
y = [0.1, 0.05, 0.6, 0.0, 0.05, 0.1, 0.0, 0.1, 0.0, 0.0] # 신경망의 출력
t = [0,0,1,0,0,0,0,0,0,0] # 정답 레이블

sum_of_squares(np.array(y), np.array(t)) # 0.09750000000000003

0.09750000000000003

In [12]:
# 교차 엔트로피 오차

def cross_entropy_error(y, t):
  delta = 1e-7
  return -np.sum(t * np.log(y + delta)) # 아주 작은 delta를 더해줌으로써 np.log(0)이 -inf가 되는 것을 방지

In [13]:
y = [0.1, 0.05, 0.6, 0.0, 0.05, 0.1, 0.0, 0.1, 0.0, 0.0] # 신경망의 출력
t = [0,0,1,0,0,0,0,0,0,0] # 정답 레이블

cross_entropy_error(np.array(y), np.array(t)) # 0.510825457099338

0.510825457099338

In [14]:
delta = 1e-7
a = -(1*np.log(0.6+delta)) # 정답 레이블이 1일 때의 교차 엔트로피 오차
print(a)

0.510825457099338


### 미니배치 학습

In [15]:
import sys, os
sys.path.append('C:/Users/KimDongyoung/Desktop/Github/my_git/mygit/DEEPLEARNING/퍼셉트론/mnist.py')
import pickle
from mnist import load_mnist
import tensorflow as tf

In [16]:
# MNIST 데이터셋 불러오기
mnist = tf.keras.datasets.mnist
(x_train, t_train), (x_test, t_test) = mnist.load_data()  # x_train, t_train: 훈련 데이터, x_test, t_test: 테스트 데이터

# 데이터 정규화
x_train, x_test = x_train / 255.0, x_test / 255.0

# 데이터 형태 변환
x_train = x_train.reshape(-1, 784)
x_test = x_test.reshape(-1, 784)

In [43]:
train_size = x_train.shape[0]
batch_size = 10
batch_mask = np.random.choice(train_size, batch_size)
x_batch = x_train[batch_mask]
t_batch = t_train[batch_mask]

In [31]:
test_size = x_test.shape[0]
batch_mask = np.random.choice(test_size, batch_size)
x_batch = x_test[batch_mask]
t_batch = t_test[batch_mask]


In [44]:
x_batch.shape, t_batch.shape

((10, 784), (10,))

In [47]:
# 미니배치용 교차 엔트로피 오차 구현
# 정답 레이블이 원-핫 인코딩이 아닌 숫자 레이블로 주어졌을 때

def cross_entropy_error(y, t):
    if y.ndim == 1:
        t = t.reshape(1, t.size) # t는 실제 정답 레이블
        y = y.reshape(1, y.size) # y는 신경망의 출력

    batch_size = y.shape[0]
    return -np.sum(np.log(y[np.arange(batch_size), t] + 1e-7)) / batch_size # np.arange(batch_size)는 0부터 batch_size-1까지 배열 생성

In [48]:
cross_entropy_error(np.array(x_batch), np.array(t_batch)) # 0.510825457099338

16.118095650958317

### 수치미분

In [50]:
def numerical_diff(f, x):
  h = 1e-4
  return (f(x+h) - f(x-h)) / (2*h)

def function_1(x):
  return 0.01**2 + 0.1*x

In [51]:
numerical_diff(function_1, 5) # 0.1999999999990898

0.0999999999995449

### 편미분

In [52]:
def function_2(x):
  return x[0]**2 + x[1]**2  # x_0^2 + x_1^2

In [57]:
# x0 = 3, x1 = 4일 때, x0에 대한 편미분

def function_tmp1(x0):
  return x0*x0 + 4.0**2.0

numerical_diff(function_tmp1, 3.0) # 6.00000000000378

6.00000000000378

In [58]:
# x0 = 3, x1 = 4일 때, x1에 대한 편미분

def function_tmp2(x1):
  return 3.0**2.0 + x1*x1

numerical_diff(function_tmp1, 4.0) # 7.999999999999119

7.999999999999119

In [59]:
# 편미분을 동시에 계산

def numerical_gradient(f, x):
  h = 1e-4
  grad = np.zeros_like(x)

  for idx in range(x.size):
    tmp_val = x[idx]

    # f(x+h) 계산
    x[idx] = tmp_val + h
    fxh1 = f(x)

    # f(x-h) 계산
    x[idx] = tmp_val - h
    fxh2 = f(x)

    grad[idx] = (fxh1 - fxh2) / (2*h)   # f(x+h) - f(x-h) / 2h
    x[idx] = tmp_val

  return grad

In [60]:
numerical_gradient(function_2, np.array([3.0, 4.0])) # array([6., 8.])

array([6., 8.])

### 경사하강법 

In [61]:
def gradient_descent(f,init_x, lr=0.01, step_num=100):
  x = init_x

  for i in range(step_num):
    grad = numerical_gradient(f, x)
    x -= lr * grad

  return x

In [62]:
def function_2(x):
  return x[0]**2 + x[1]**2

init_x = np.array([-3.0, 4.0])
gradient_descent(function_2, init_x=init_x, lr=0.1, step_num=100) # (-3,4)로 시작하여 (0,0)으로 수렴, 실제로 최솟값은 (0,0)이므로 정확하게 수렴

array([-6.11110793e-10,  8.14814391e-10])

### 신경망에서의 기울기 

In [83]:
# 소프트맥스 함수 구현시 주의점: 오버플로 방지
# 지수 함수는 쉽게 아주 큰 값을 내뱉어버릴 수 있기 때문에 이를 방지하기 위해 입력 신호 중 최댓값을 빼주는 방법을 사용
# 가령 np.exp(10)은 22026.465794806718이지만, np.exp(1000)은 inf를 반환
# np.max(x)는 입력 신호 중 최댓값을 반환
def softmax(x):
    if x.ndim == 2:
        x = x - np.max(x, axis=1, keepdims=True) # 오버플로 대책
        x = np.exp(x)
        x /= np.sum(x, axis=1, keepdims=True)
    elif x.ndim == 1:
        x = x - np.max(x) # 오버플로 대책
        x = np.exp(x) / np.sum(np.exp(x))
    return x

# 평균 손실함수
def cross_entropy_error(y, t):
    if y.ndim == 1:
        t = t.reshape(1, t.size) # t는 실제 정답 레이블
        y = y.reshape(1, y.size) # y는 신경망의 출력

    batch_size = y.shape[0]
    return -np.sum(t * np.log(y + 1e-7)) / batch_size  # np.log(0)이 -inf가 되는 것을 방지하기 위해 아주 작은 delta(1e-7)를 더해줌

# 수치 미분
def numerical_gradient(f, x):
    h = 1e-4 # 0.0001, 
    grad = np.zeros_like(x)
    
    it = np.nditer(x, flags=['multi_index'], op_flags=['readwrite'])
    # np.nditer: 다차원 배열을 순차적으로 접근할 수 있게 해주는 이터레이터
    while not it.finished: # 이터레이터의 끝까지 반복
        idx = it.multi_index
        tmp_val = x[idx]
        
        # f(x+h) 계산
        x[idx] = tmp_val + h
        fxh1 = f(x)
        
        # f(x-h) 계산
        x[idx] = tmp_val - h
        fxh2 = f(x)
        
        grad[idx] = (fxh1 - fxh2) / (2*h)
        x[idx] = tmp_val
        it.iternext()
        
    return grad

In [84]:
class simpleNet:
  def __init__(self):
    self.W = np.random.randn(2,3) # 가중치 랜덤으로 초기화, 2*3 행렬
    
  def predict(self, x):
    return np.dot(x, self.W)
  
  def loss(self, x, t):
    z = self.predict(x)
    y = softmax(z)
    loss = cross_entropy_error(y, t)  # 손실함수
    
    return loss

In [85]:
net = simpleNet()
print(net.W) # 가중치 매개변수, 랜덤으로 초기화
x = np.array([0.6, 0.9])
p = net.predict(x)
print(p)

[[ 0.44386323  0.33367433  1.49407907]
 [-0.20515826  0.3130677  -0.85409574]]
[0.0816755  0.48196553 0.12776128]


In [86]:
t = np.array([0, 0, 1]) # 정답 레이블
net.loss(x, t)

1.217877225737111

In [87]:
def f(W):
    return net.loss(x, t)

dW = numerical_gradient(f, net.W)
print(dW)

[[ 0.16951916  0.25296623 -0.42248539]
 [ 0.25427874  0.37944935 -0.63372809]]


### 2층 신경망 클래스 구현하기 

In [88]:
# 퍼셉트론과 달리 신경망은 활성화 함수로 시그모이드 함수를 사용한다.

def sigmoid(x):
  return 1 / (1 + np.exp(-x))

In [92]:
class TwoLayerNet:
  def __init__(self, input_size, hidden_size, output_size, weight_init_std =0.01): # weight_init_std: 가중치 초기화 시 정규분포의 스케일
    self.params = {}
    # 평균이 0이고 표준편차가 weight_init_std인 정규분포를 따르는 가중치를 생성하여 초기화
    self.params['W1'] = weight_init_std * np.random.randn(input_size, hidden_size) # 1번째 층의 가중치, 예를 들어 input size(입력)가 784이고 hidden size(은닉층)가 100이면 784*100의 가중치 행렬이 생성
    self.params['b1'] = np.zeros(hidden_size) # 1번째 층의 편향, 이 배열의 모든 원소는 0으로 초기화
    self.params['W2'] = weight_init_std * np.random.randn(hidden_size, output_size) # 2번째 층의 가중치, output size은 출력층의 뉴런 수
    self.params['b2'] = np.zeros(output_size)
  
  def predict(self,x):
    W1, W2 = self.params['W1'], self.params['W2']
    b1, b2 = self.params['b1'], self.params['b2']
    
    # 입력층에서 1층으로의 신호 전달
    a1 = np.dot(x, W1) + b1  
    z1 = sigmoid(a1) # 활성화 함수로 시그모이드 함수 사용
    
    # 1층에서 출력층으로의 신호 전달
    a2 = np.dot(z1, W2) + b2
    y = softmax(a2) # 출력층의 활성화 함수로 소프트맥스 함수 사용
    
    return y
  
  def loss(self, x, t): # x: 입력 데이터, t: 정답 레이블
    y = self.predict(x) # 신경망의 출력(예측값)
    
    # 손실함수로 교차 엔트로피 오차 사용
    return cross_entropy_error(y, t) # 미니배치 처리를 고려하여 평균 손실함수 반환
  
  def accuracy(self, x, t):
    y = self.predict(x)
    y = np.argmax(y, axis=1) # 각 데이터의 예측 출력값 중 가장 큰 값의 인덱스 반환
    
    if t.ndim != 1: # 정답 레이블이 원-핫 인코딩이 아닌 경우
      t = np.argmax(t, axis=1)  # 인덱스 형태로 변환
    accuracy = np.sum(y == t) / float(x.shape[0])
    return accuracy
  
  # 수치 미분을 사용한 기울기 계산
  # x: 입력 데이터, t: 정답 레이블
  def numerical_gradient(self, x, t):
    loss_W = lambda W: self.loss(x, t) # 손실함수의 W에 대한 함수
    
    grads = {}
    grads['W1'] = numerical_gradient(loss_W, self.params['W1']) # 수치 미분을 활용해 W1 기울기를 계산, 갱신
    grads['b1'] = numerical_gradient(loss_W, self.params['b1'])
    grads['W2'] = numerical_gradient(loss_W, self.params['W2'])
    grads['b2'] = numerical_gradient(loss_W, self.params['b2'])
    
    return grads

In [ ]:
# 신경망 생성
net = TwoLayerNet(input_size=784, hidden_size=100, output_size=10)
net.params['W1'].shape, net.params['b1'].shape, net.params['W2'].shape, net.params['b2'].shape

x = np.random.rand(100, 784) # 더미 입력 데이터(100장 분량)
t = np.random.rand(100, 10) # 더미 정답 레이블(100장 분량)

11.32676196882358 0.12


In [101]:
# mnist 데이터셋 불러오기
(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)

Done
Done
Done
Done
Converting train-images-idx3-ubyte.gz to NumPy Array ...
Done
Converting train-labels-idx1-ubyte.gz to NumPy Array ...
Done
Converting t10k-images-idx3-ubyte.gz to NumPy Array ...
Done
Converting t10k-labels-idx1-ubyte.gz to NumPy Array ...
Done
Creating pickle file ...
Done!


In [102]:
x_train.shape, t_train.shape, x_test.shape, t_test.shape

((60000, 784), (60000, 10), (10000, 784), (10000, 10))

In [109]:
iters_num = 5  # 반복 횟수
train_size = x_train.shape[0]
batch_size = 100  # 미니배치 크기
learning_rate = 0.1

train_loss_list = []

network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

for i in range(iters_num):
    batch_mask = np.random.choice(train_size, batch_size)
    x_batch = x_train[batch_mask]
    t_batch = t_train[batch_mask]
    
    # 기울기 계산
    grad = network.numerical_gradient(x_batch, t_batch)
    
    # 매개변수 갱신
    for key in ('W1', 'b1', 'W2', 'b2'):
        network.params[key] -= learning_rate * grad[key]
        
    # 학습 경과 기록
    loss = network.loss(x_batch, t_batch)
    train_loss_list.append(loss)

In [108]:
# 정확도 계산
train_acc = network.accuracy(x_train, t_train)
test_acc = network.accuracy(x_test, t_test)
train_acc, test_acc

(0.09736666666666667, 0.0982)

In [110]:
# 정확도 계산
train_acc = network.accuracy(x_train, t_train)
test_acc = network.accuracy(x_test, t_test)
train_acc, test_acc

(0.11236666666666667, 0.1135)